<a href="https://colab.research.google.com/github/kylemath/DeepEEG/blob/master/notebooks/Deep_EEG_Muse.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## DeepEEG


In [3]:
!git clone https://github.com/kylemath/DeepEEG
!chmod +x ./DeepEEG/install.sh
%cd DeepEEG
!./install.sh
from utils import *
%matplotlib inline

Cloning into 'DeepEEG'...
remote: Enumerating objects: 13, done.
remote: Counting objects: 100% (13/13), done.
remote: Compressing objects: 100% (12/12), done.
remote: Total 947 (delta 2), reused 5 (delta 1), pack-reused 934
Receiving objects: 100% (947/947), 19.62 MiB | 8.43 MiB/s, done.
Resolving deltas: 100% (489/489), done.
/content/DeepEEG/DeepEEG
Requirement already up-to-date: pip in /usr/local/lib/python3.6/dist-packages (20.2.2)


#Load Data, Concatenate


In [4]:
!git clone https://github.com/kylemath/eeg-notebooks_v0.1/
data_dir = 'visual/cueing'

subs = [101, 102, 103, 104, 105, 106, 108, 109, 110, 111, 112,
        202, 203, 204, 205, 207, 208, 209, 210, 211,
        301, 302, 303, 304, 305, 306, 307, 308, 309,
        1101, 1102, 1103, 1104, 1105, 1106, 1108, 1109, 1110,
        1202, 1203, 1205, 1206, 1209, 1210, 1211, 1215,
        1301, 1302, 1313, 
        1401, 1402, 1403, 1404, 1405,  1408, 1410, 1411, 1412, 1413, 1413, 1414, 1415, 1416]

nsesh = 2
event_id = {'LeftCue':1, 'RightCue':2}

raw = LoadMuseData(subs,nsesh,data_dir)

#subs = [ 1]
#nsesh = 1
#data_dir = 'visual/P300'
#event_names = ['Standard','Target']

#subs = [ 4]
#nsesh = 1
#data_dir = 'visual/SSVEP'
#event_names = ['f30Hz','f20Hz']

Cloning into 'eeg-notebooks_v0.1'...
remote: Enumerating objects: 2451, done.
remote: Total 2451 (delta 0), reused 0 (delta 0), pack-reused 2451
Receiving objects: 100% (2451/2451), 166.74 MiB | 12.34 MiB/s, done.
Resolving deltas: 100% (1096/1096), done.
Checking out files: 100% (632/632), done.
Loading Data
Subject number 1/63
 Session number 1/2
No files for subject with filename []
 Session number 2/2
No files for subject with filename []
Subject number 2/63
 Session number 1/2
No files for subject with filename []
 Session number 2/2
No files for subject with filename []
Subject number 3/63
 Session number 1/2
No files for subject with filename []
 Session number 2/2
No files for subject with filename []
Subject number 4/63
 Session number 1/2
No files for subject with filename []
 Session number 2/2
No files for subject with filename []
Subject number 5/63
 Session number 1/2
No files for subject with filename []
 Session number 2/2
No files for subject with filename []
Subject n

TypeError: ignored

#Run Preprocessing

**Input:  mne.raw, event_id**

**Output: mne.epochs**
```python
plot_psd=False
filter_data=True
eeg_filter_highpass=1
plot_events=False
epoch_time=(-.2,1)
baseline=(-.2,0)
rej_thresh_uV=200
rereference=False
emcp_raw=False
emcp_epochs=False
epoch_decim=1
plot_electrodes=False
plot_erp=False
```

In [ ]:
epochs = PreProcess(raw,event_id)


Filtering Data Between 1 and 30 Hz.
4583 events found
Event IDs: [ 1  2 11 12 21 22]
Remaining Trials: 1408


#Run FeatureEngineer
**Input:  mne.epochs**

**Output: deepeeg.feats**
```python
model_type='NN'
frequency_domain=False
normalization=True
electrode_median=False
wavelet_decim=1
flims=(3,30)
f_bins=20
wave_cycles=3
spect_baseline=[-1,-.5]
electrodes_out=[11,12,13,14,15]
test_split = 0.2
val_split = 0.2
random_seed=1017
watermark = False
```

In [ ]:
feats = FeatureEngineer(epochs,model_type='NN')

Constructing Time Domain Features
Normalizing X
Combined X Shape: (1408, 256, 4)
Combined Y Shape: (1408,)
Y Example (should be 1s & 0s): [1 1 0 0 0 0 0 1 0 0]
X Range: -12.106191:9.781672
Input Shape: (256, 4)
x_train shape: (844, 256, 4)
844 train samples
282 test samples
282 validation samples
Class Weights: [0.97011494 1.03178484]


# Run CreateModel

**Input: deepeeg.feats**

**Output: deepeeg.model, deepeeg.encoder**

```python
units=[16,8,4,8,16]
dropout=.25
batch_norm=True
filt_size=3
pool_size=2
```

In [ ]:
model, _ = CreateModel(feats)

# TrainTestVal

**Input: deepEEG.model, deepEEG.feats**

```python
batch_size=2
train_epochs=20
show_plots=True
```

In [ ]:
TrainTestVal(model, feats)